In [4]:
# !pip install awswrangler

In [8]:
from urllib.parse import unquote_plus
import awswrangler as wr 
import pandas as pd
import numpy as np
import boto3

bucket = 'poc-2209-volkswagen/marketstatus-import'
key = 'raw/2022/08/KAIDA PC Registration(Aug., 2022)_Tesla.xls'
_, year, month, filename = key.split('/')

In [9]:
path = f's3://{bucket}/{key}'
df = wr.s3.read_excel(path,
    header=3,
    skipfooter=1,
    usecols='A:E,G:R',
    names=['Company', 'Brand', 'Model', 'cc', 'Price',1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,],
)

In [10]:
company_meta_path = 's3://poc-2209-volkswagen/marketstatus-metas/company/'
company = wr.s3.read_csv(company_meta_path)

brand_meta_path = 's3://poc-2209-volkswagen/marketstatus-metas/brand/'
brand = wr.s3.read_csv(brand_meta_path)

df = df[df['Model'].notnull()]
df['Company'] = df['Company'].str.replace(' \n ', ' ')

company_new = []
for c in df['Company']:
    if company['Company_raw'].isin([c]).any():
        company_new.append(company.loc[company['Company_raw'] == c, 'Company'].item())
    elif pd.isna(c):
        company_new.append(np.NaN)
    else:
        raise ValueError(f'No such value in meta table: {c}')
df['Company'] = company_new

brand_new = []
for c in df['Brand']:
    if brand['Brand_raw'].isin([c]).any():
        brand_new.append(brand.loc[brand['Brand_raw'] == c, 'Brand'].item())
    elif pd.isna(c):
        brand_new.append(np.NaN)
    else:
        raise ValueError(f'No such value in meta table: {c}')
df['Brand'] = brand_new

df[['Company', 'Brand']] = df[['Company', 'Brand']].fillna(method='ffill')

df = pd.melt(df, id_vars=['Company', 'Brand', 'Model', 'cc', 'Price'], value_vars=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,], var_name='Month', value_name='Qty')
df['Qty'] = df['Qty'].astype(float)

df['Year'] = int(year)

In [55]:
########################################################################################################
master_meta_path = 's3://poc-2209-volkswagen/marketstatus-metas/master/'
master = wr.s3.read_csv(master_meta_path, encoding='euckr')

In [58]:
model_new = []
start = time.time()
for c in df['Model']:
    if (c in master['Model'].values) or (pd.isna(c)):
        pass
    else:
        model_new.append(c)            
        
time.time() - start

0.5683937072753906

In [68]:
os.environ['snsARN'] = 'arn:aws:sns:ap-northeast-2:257832939543:temp_marketstatus-lambda-error-topic'

In [72]:
import os
from botocore.exceptions import ClientError

if len(model_new) > 0:
    sns_arn = os.environ['snsARN']  # Getting the SNS Topic ARN passed in by the environment variables.
    snsclient = boto3.client('sns')
    try:
        message = ""
        message += "\nModel value error  summary" + "\n\n"
        message += "##########################################################\n"
        message += "# Number of model not included:- " + str(len(model_new)) + "\n"
        message += "# Model list:- " + str(model_new) + "\n"
        message += "##########################################################\n"

        # Sending the notification...
        snsclient.publish(
            TargetArn=sns_arn,
            Subject=f'Execution error for Lambda',
            Message=message
        )
    except ClientError as e:
        logger.error("An error occured: %s" % e)
